In [4]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import efficientnet
import efficientnet.tfkeras as efn
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Dropout, GlobalAveragePooling2D, Input
import tensorflow_addons as tfa
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import matplotlib.pyplot as plt
import cv2

In [25]:
train_dir = '../../data/cat_dog_small/train'
valid_dir = '../../data/cat_dog_small/validation'

IMAGE_SIZE = 256
BATCH_SIZE = 8
LEARNING_RATE = 5e-5

def generate_preprocessing(img):
    img = cv2.resize(img, (IMAGE_SIZE, IMAGE_SIZE))
    return img

train_datagen = ImageDataGenerator(rescale=1/255.0,
                                   rotation_range=40,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   preprocessing_function=generate_preprocessing,
                                   fill_mode='nearest')      

valid_datagen = ImageDataGenerator(rescale=1/255.0,
                                   preprocessing_function=generate_preprocessing)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    classes=['cats', 'dogs'],                                                    
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='binary')

valid_generator = valid_datagen.flow_from_directory(valid_dir,
                                                    classes=['cats', 'dogs'],                                                    
                                                    batch_size=BATCH_SIZE,
                                                    class_mode='binary',
                                                    shuffle=False)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [27]:
pretrained_network = efn.EfficientNetB4(
    weights='imagenet',
    include_top=False,
    input_shape=(IMAGE_SIZE,IMAGE_SIZE,3)
)

pretrained_network.trainable = False  # 학습이 안되게끔 동결!!!!

model = Sequential()

model.add(pretrained_network)

model.add(GlobalAveragePooling2D())
model.add(Dense(units=1,
                activation='sigmoid'))

# Early Stopping
es = EarlyStopping(monitor='val_loss',
                   mode='auto',
                   patience=5,
                   verbose=1)

# Checkpoint
model_checkpoint = './{epoch:06d}-{val_acc:0.6f}-{acc:0.6f}.ckpt'

checkpointer = ModelCheckpoint(
    filepath=model_checkpoint,
    verbose=1,
    period=2,
    save_best_weights=True,
    mode='auto',
    monitor='val_acc'
)

model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(train_generator,
                    steps_per_epoch=(2000 // BATCH_SIZE),
                    epochs=30,
                    validation_data=valid_generator,
                    validation_steps=(1000 // BATCH_SIZE),
                    callbacks=[es, checkpointer],
                    verbose=1)

Epoch 1/30
250/250 [==============================] - 49s 157ms/step - loss: 0.6138 - acc: 0.6885 - val_loss: 0.5019 - val_acc: 0.8750
Epoch 2/30
250/250 [==============================] - ETA: 0s - loss: 0.5009 - acc: 0.8265
Epoch 2: saving model to .\000002-0.931000-0.826500.ckpt


INFO:tensorflow:Assets written to: .\000002-0.931000-0.826500.ckpt\assets


INFO:tensorflow:Assets written to: .\000002-0.931000-0.826500.ckpt\assets


250/250 [==============================] - 81s 326ms/step - loss: 0.5009 - acc: 0.8265 - val_loss: 0.3846 - val_acc: 0.9310
Epoch 3/30
250/250 [==============================] - 39s 153ms/step - loss: 0.4296 - acc: 0.8585 - val_loss: 0.3129 - val_acc: 0.9420
Epoch 4/30
250/250 [==============================] - ETA: 0s - loss: 0.3747 - acc: 0.8860
Epoch 4: saving model to .\000004-0.949000-0.886000.ckpt


INFO:tensorflow:Assets written to: .\000004-0.949000-0.886000.ckpt\assets


...

INFO:tensorflow:Assets written to: .\000028-0.988000-0.922500.ckpt\assets


250/250 [==============================] - 94s 375ms/step - loss: 0.1940 - acc: 0.9225 - val_loss: 0.0779 - val_acc: 0.9880
Epoch 29/30
250/250 [==============================] - 47s 187ms/step - loss: 0.1837 - acc: 0.9290 - val_loss: 0.0762 - val_acc: 0.9880
Epoch 30/30
250/250 [==============================] - ETA: 0s - loss: 0.1953 - acc: 0.9295
Epoch 30: saving model to .\000030-0.988000-0.929500.ckpt


INFO:tensorflow:Assets written to: .\000030-0.988000-0.929500.ckpt\assets


INFO:tensorflow:Assets written to: .\000030-0.988000-0.929500.ckpt\assets


250/250 [==============================] - 91s 364ms/step - loss: 0.1953 - acc: 0.9295 - val_loss: 0.0749 - val_acc: 0.9880
